In [1]:
from langchain_openai.chat_models import ChatOpenAI
from langgraph.graph import StateGraph
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
tools = [TavilySearchResults(max_results=1)]

In [6]:
from langgraph.prebuilt import ToolExecutor
tool_executor = ToolExecutor(tools)

In [7]:
model = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")

In [9]:
from pprint import pprint

In [11]:
from langchain.tools.render import format_tool_to_openai_function
functions = [format_tool_to_openai_function(tool) for tool in tools]
pprint(functions)

[{'description': 'A search engine optimized for comprehensive, accurate, and '
                 'trusted results. Useful for when you need to answer '
                 'questions about current events. Input should be a search '
                 'query.',
  'name': 'tavily_search_results_json',
  'parameters': {'properties': {'query': {'description': 'search query to look '
                                                         'up',
                                          'type': 'string'}},
                 'required': ['query'],
                 'type': 'object'}}]


In [12]:
model = model.bind_functions(functions)

In [28]:
# state graph is the main type of graph in langgraph. The graph is parameterized by a state objest that it passes around to each node. 
# Each node then returns ooperations to update that state. This operations can either be SET a specific attribute on the graph,or ADD to an existing atribute

# in this example the state will just be a list of messages. We want each noe to add messages to that list. Therefor we would define our agent state as a TypedDict 
# with a single attribute of messages which is a list of messages

from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

### Defining the Nodes within our LangGraph

A node can be either a function or a langchain runnable (chain, agent, or anything that implements the runnable class)
In this example the two main nodes we need are:

- Agent node: Decides what action if any should be taken
- Function to invoke tools: if the agent decides to take an action, this node will then execute that action

We will also need to define some edges. Edges may be conditional. The reason they are conditional is that based on the output of a node, one of several paths may be taken. The path that is taken is not known ahead of time until the source node is executed. 

- Conditional Edge: after the agent is called, we should either:
  - if the agent decides to take an action, then the function to invoke the tools should be called. 
  - if the aegnt said that it was finished, then it should finish
- Normal edges: After the tool is invoked, it should always go back to the agent to decide what to do next

In [29]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

In [50]:
def should_continue(state): # the branch conditions take the state as the only parameter
    messages = state["messages"]
    last_message:BaseMessage =  messages[-1]
    # if there is no function call, then we finish
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    else:
        return "continue"
    
    
def call_model(state): # all nodes also take only the state as input
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}


def call_tool(state): # all nodes also take only the state as input
    messages = state["messages"]
    last_message:BaseMessage = messages[-1]
    
    action = ToolInvocation(
        tool=last_message.additional_kwargs["tool_calls"][0]["function"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["tool_calls"][0]["function"]["arguments"])
    )
    # we call the tool executoro and get back a response
    response = tool_executor.invoke(action)
    
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # we return a lsist, becasuse this will be added to th existing list
    return {"messages": [function_message]}
    

## Define the graph
Now we can put everything together to define the graph

In [51]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(schema=AgentState)

# define the nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

# define and entry point for our graph
workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    # first we define the starting point of the conditional edge
    "agent", 
    # this determines if we should continue the decision to be taken
    should_continue, 
    # this defines what outcome we should proceed with based on the output of `should_continue`
    {
        "continue": "action", 
        "end": END # end is also a node added to every graph
    }
    )

workflow.add_edge("action", "agent")


In [52]:
app = workflow.compile()

In [53]:
pprint(app.to_json())

{'id': ['langgraph', 'pregel', 'Pregel'],
 'lc': 1,
 'repr': "Pregel(nodes={'agent': ChannelInvoke(bound=RunnableLambda(...)\n"
         '| RunnableLambda(call_model)\n'
         "| ChannelWrite(channels=[('agent', None, False), ('messages', "
         "RunnableLambda(...), False)]), config={'tags': []}, channels={None: "
         "'agent:inbox'}, triggers=['agent:inbox']), 'action': "
         'ChannelInvoke(bound=RunnableLambda(...)\n'
         '| RunnableLambda(call_tool)\n'
         "| ChannelWrite(channels=[('action', None, False), ('messages', "
         "RunnableLambda(...), False)]), config={'tags': []}, channels={None: "
         "'action:inbox'}, triggers=['action:inbox']), 'agent:edges': "
         'ChannelInvoke(bound=RunnableLambda(_read)\n'
         "| RunnableLambda(runnable), config={'tags': ['langsmith:hidden']}, "
         "channels={None: 'agent'}, triggers=['agent']), 'action:edges': "
         'ChannelInvoke(bound=RunnableLambda(_read)\n'
         "| ChannelWrite(c

In [54]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in sf'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xogmog8iuykyk4iv75i4szy5', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}),
  FunctionMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "Weather in San Francisco is {\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1707957221, \'localtime\': \'2024-02-14 16:33\'}, \'current\': {\'last_updated_epoch\': 1707957000, \'last_updated\': \'2024-02-14 16:30\', \'temp_c\': 12.8, \'temp_f\': 55.0, \'is_day\': 1, \'condition\': {\'text\': \'Light rain\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/296.png\', \'code\': 1183}, \'wind_mph\': 16.1, \'wind_kph\': 25.9, \'wind_degree\': 160, \'wind_dir\': 

In [55]:
inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
for output in app.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_hjb479k7vneuj3msnjcf3axp', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]})]}

---

Output from node 'action':
---
{'messages': [FunctionMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "Weather in San Francisco is {\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1707957221, \'localtime\': \'2024-02-14 16:33\'}, \'current\': {\'last_updated_epoch\': 1707957000, \'last_updated\': \'2024-02-14 16:30\', \'temp_c\': 12.8, \'temp_f\': 55.0, \'is_day\': 1, \'condition\': {\'text\': \'Light rain\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/296.png\', \'code\': 1183}, \'wind_mph\': 16.1, \'wind_kph\': 25.9, \'wind_de